In [6]:
# TODO: Dirty hack to import from sibling dir. Put wahlrecht_polling_firms.py into the same folder as this file eventually.
import sys
import os
sys.path.append(os.path.abspath('../Backend'))
import wahlrecht_polling_firms

import numpy as np
import pandas as pd

data_dict = wahlrecht_polling_firms.get_tables()

In [7]:
parties = ['CDU/CSU', 'SPD', 'GRÜNE', 'FDP', 'LINKE', 'AfD', 'Sonstige']

In [17]:
def _normalize_to_one(x):
    """Normalize an array so that its sum is 1."""
    return x / np.sum(x)

np.sum(_normalize_to_one(np.linspace(0, 10)))

1.0

In [9]:
def _p2f(x):
    """Convert string with percent to float."""
    return float(x.strip('%').replace(',', '.')) / 100

In [18]:
def _prediction_to_dataframe(prediction):
    """Wrap an array with the predictions into a dataframe containing the party names."""
    return pd.DataFrame(data=[prediction], columns=parties)

_prediction_to_dataframe(range(len(parties)))

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0,1,2,3,4,5,6


In [11]:
def average(n_last=5):
    """Average the last `n_last` polls from all polling firms."""
    prediction = np.zeros(len(parties))
    for df in data_dict.values():
        for i in range(n_last):
            # TODO: Contains some fixes to convert strings to floats. Do this in the data already.
            try:
                results = np.array(list(map(_p2f, df[parties].ix[i])))
                # TODO: Polls from different polling firms have different time spacing. Take this into account. 
                prediction += results
            except ValueError as e:
                print(e)  # TODO: Probably just a conversion error. Remove this once the data is fixed.
    return _prediction_to_dataframe(_normalize_to_one(prediction))

average()

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.355429,0.281143,0.076143,0.067714,0.085714,0.091571,0.042286


In [12]:
def weighted_average(n_last=5):
    """Average the last `n_last` polls from all polling firms, weighted by the number of participants."""
    prediction = np.zeros(len(parties))
    for df in data_dict.values():
        for i in range(n_last):
            # TODO: Contains some fixes to convert strings to floats. Do this in the data already.
            try:
                results = np.array(list(map(_p2f, df[parties].ix[i])))
                num_people = float(df['Befragte'].ix[i].replace('.', '').replace('T • ', '').replace('O • ', ''))
                # TODO: Polls from different polling firms have different time spacing. Take this into account. 
                prediction += results * num_people
            except ValueError as e:
                print(e)  # TODO: Probably just a conversion error. Remove this once the data is fixed.
    return _prediction_to_dataframe(_normalize_to_one(prediction))

weighted_average()

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.357701,0.280034,0.074463,0.068739,0.08645,0.089384,0.043228


In [13]:
def latest():
    """Average the latest polls from all polling firms."""
    return average(n_last=1)

latest()

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.373571,0.271429,0.07,0.079286,0.084286,0.078571,0.042857


In [14]:
def weighted_latest():
    """Average the latest polls from all polling firms, weighted by the number of participants."""
    return weighted_average(n_last=1)

weighted_latest()

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.375411,0.267579,0.069126,0.080299,0.086151,0.077244,0.04419


In [15]:
def decay(decay_factor=0.9):
    """Average all polls from all polling firms, where polls further back are weighted less (exponential decay)."""
    prediction = np.zeros(len(parties))
    for df in data_dict.values():
        for i in range(len(df)):
            # TODO: Contains some fixes to convert strings to floats. Do this in the data already.
            try:
                results = np.array(list(map(_p2f, df[parties].ix[i])))
                # TODO: Polls from different polling firms have different time spacing. Take this into account. 
                prediction += results * decay_factor**(i+1)
            except ValueError as e:
                print(e)  # TODO: Probably just a conversion error. Remove this once the data is fixed.
    return _prediction_to_dataframe(_normalize_to_one(prediction))

decay()

could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: 'PIR 3 %Sonst. 5 '
could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: 'PIRATEN 3 %Sonst. 3 '
could not convert string to float: 'PIRATEN 2.2 %Sonst. 4.0 '
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to float: '–'
could not convert string to 

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.351692,0.266669,0.087518,0.06299,0.087306,0.098836,0.04499


In [16]:
def weighted_decay(decay_factor=0.9):
    """Average all polls from all polling firms, where polls further back are weighted less (exponential decay) and each poll is weighted by the number of participants."""
    prediction = np.zeros(len(parties))
    for df in data_dict.values():
        for i in range(len(df)):
            # TODO: Contains some fixes to convert strings to floats. Do this in the data already.
            try:
                results = np.array(list(map(_p2f, df[parties].ix[i])))
                num_people = float(df['Befragte'].ix[i].replace('.', '').replace('T • ', '').replace('O • ', ''))
                # TODO: Polls from different polling firms have different time spacing. Take this into account. 
                prediction += results * decay_factor**(i+1) * num_people
            except ValueError as e:
                print(e)  # TODO: Probably just a conversion error. Remove this once the data is fixed.
    return _prediction_to_dataframe(_normalize_to_one(prediction))

weighted_decay()

could not convert string to float: '≈1400'
could not convert string to float: '≈1400'
could not convert string to float: '≈1400'
could not convert string to float: '≈1200'
could not convert string to float: '≈1400'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: '?'
could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: 'PIR 3 %Sonst. 5 '
could not convert string to float: 'PIR 2.2 %Sonst. 4.0 '
could not convert string to float: '–'
could not c

,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige
0,0.350253,0.270133,0.084474,0.063716,0.087522,0.098235,0.045666
